In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np

data1 = pd.read_csv("CSV_EXCEL\\submissions_data_train.csv")
data1

,step_id,timestamp,submission_status,user_id
0,31971,1434349275,correct,15853
1,31972,1434348300,correct,15853
2,31972,1478852149,wrong,15853
3,31972,1478852164,correct,15853
4,31976,1434348123,wrong,15853
...,...,...,...,...
509099,120745,1501946959,wrong,2615
509100,120745,1503059927,correct,13177
509101,120745,1506348153,wrong,15253
509102,120745,1523035316,wrong,1424


In [2]:
data2 = pd.read_csv("CSV_EXCEL\\event_data_train.csv")
data2

#array(['viewed', 'passed', 'discovered', 'started_attempt'], dtype=object)

#discovered - пользователь перешел на стэп
#viewed - просмотр шага,
#started_attempt - начало попытки решить шаг, ранее нужно было явно нажать на кнопку - начать решение, перед тем как приступить к решению практического шага
#passed - удачное решение практического шага

,step_id,timestamp,action,user_id
0,32815,1434340848,viewed,17632
1,32815,1434340848,passed,17632
2,32815,1434340848,discovered,17632
3,32811,1434340895,discovered,17632
4,32811,1434340895,viewed,17632
...,...,...,...,...
3480698,33375,1526772685,viewed,11677
3480699,32815,1526772750,viewed,5198
3480700,32815,1526772750,discovered,5198
3480701,32815,1526772750,passed,5198


In [3]:
data2  = data2[(data2.action == "started_attempt") | (data2.action == "viewed")|(data2.action == "discovered")]
data2  = data2.groupby(["step_id","action"], as_index=False).aggregate({"user_id": "count"})
data2.sort_values("step_id")[data2.step_id == 31978]


C:\Users\Валерий\AppData\Local\Temp\ipykernel_7904\1808240983.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data2.sort_values("step_id")[data2.step_id == 31978]


,step_id,action,user_id
32,31978,discovered,6684
33,31978,started_attempt,7611
34,31978,viewed,20695


In [4]:
data2.groupby(["step_id"]).count().sort_values("user_id")

,action,user_id
step_id,,
30456,2,2
33378,2,2
33379,2,2
33382,2,2
33417,2,2
...,...,...
33675,3,3
33674,3,3
33673,3,3


In [5]:
data11 = data1.groupby(["submission_status", "step_id"], as_index=False).aggregate({"user_id": "count"}).rename(columns={"user_id" : "count_user_id"})
data11

,submission_status,step_id,count_user_id
0,correct,31971,6257
1,correct,31972,6434
2,correct,31976,6580
3,correct,31977,6800
4,correct,31978,6094
...,...,...,...
147,wrong,33992,2037
148,wrong,33994,2220
149,wrong,33998,859
150,wrong,34041,1243


In [6]:
data11_wrong_correct = data11[data11["submission_status"] == "wrong"].drop(["submission_status"], axis=1).rename(columns={"count_user_id": "count_wrong_user_id"})
data11_wrong_correct = data11_wrong_correct.merge(data11[data11["submission_status"] == "correct"].drop(["submission_status"], axis=1).rename(columns={"count_user_id": "count_correct_user_id"}))
data11_wrong_correct = data11_wrong_correct.set_index(data11_wrong_correct.step_id).drop(["step_id"], axis=1)

#my_list=[(abs(data11_wrong_correct.loc[step].count_wrong_user_id - data11_wrong_correct.loc[step].count_correct_user_id)) for step in data11_wrong_correct.index]

my_list=[(abs(step[0] - step[1])) for step in data11_wrong_correct.values]
     
data11_wrong_correct["result"] = my_list

data11_wrong_correct.sort_values("result", inplace=True)

data11_wrong_correct[data11_wrong_correct.result == data11_wrong_correct.result.max()]

,count_wrong_user_id,count_correct_user_id,result
step_id,,,
31978,16084,6094,9990


In [7]:
data11_wrong_correct.index

Index([ 33534,  33734,  33540,  33332,  33674,  33488,  33539,  33684,  34041,
        33983,  31977,  33079,  33732,  33686,  33351,  33874,  33890,  31983,
        33998,  33536,  33367,  31971,  33992,  31976,  33731,  33676,  33420,
        33984,  33413,  32763,  33418,  33681,  33994,  33487,  33480,  33669,
        33482,  33538,  32795,  32190,  32089,  32810,  33677,  31979,  33362,
        33415,  31986,  33685,  33439,  31972,  31988,  32173,  31991,  33350,
        32198,  33673,  33334,  32174,  32219, 120745,  31981,  32796,  33675,
        32244,  32812,  33025,  32809,  32175,  33713,  32075,  32206,  33481,
        32031,  32929,  32202,  31978],
      dtype='int64', name='step_id')

In [8]:
%%timeit # для точного измерения времени выполнения небольшого фрагмента кода
data11_wrong_correct.apply('mean')

237 μs ± 8.93 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
%%timeit
data11_wrong_correct.apply(np.mean)

315 μs ± 6.83 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
%%timeit
data11_wrong_correct.mean(axis=0)

184 μs ± 5.4 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [11]:
%%timeit
data11_wrong_correct.describe().loc['mean']

3.27 ms ± 134 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
